In [1]:
!pip install transformers datasets torch accelerate transformers[torch]

In [2]:
# Import necessary libraries
import pandas as pd

import torch


# Replace this with your actual CSV file path
file_name = 'Soil_merged.csv'

# Load the CSV into a DataFrame
data = pd.read_csv(file_name)

# Display the first few rows
data.head()

# import torch
print(torch.__version__)


2.6.0+cu124


In [3]:
# Select relevant satellite feature columns
satellite_features = [
    'polarization_diff_s1', 'ndvi_s2', 'bsi_s2', 'osavi_s2',
    'gndvi_s2', 'sr1_s2', 'sr2_s2', 'ndwi_s2', 'mean_nir_band_s2'
]

# Create a new column with string representations of the satellite data
data['input_string'] = data[satellite_features].apply(lambda row: ', '.join(
    [f"{col}: {row[col]:.4f}" for col in satellite_features]), axis=1)

# Append the target variable 'P' to each string
data['input_string'] = data['input_string'] + ' => P: ' + data['p'].astype(str)

# Display a few examples of the input strings
data[['input_string']].head()


,input_string
0,"polarization_diff_s1: 17.3009, ndvi_s2: 0.1428..."
1,"polarization_diff_s1: 21.5554, ndvi_s2: 0.1454..."
2,"polarization_diff_s1: 22.1325, ndvi_s2: 0.1274..."
3,"polarization_diff_s1: 18.1145, ndvi_s2: 0.1515..."
4,"polarization_diff_s1: 10.7781, ndvi_s2: 0.1496..."


In [4]:
from datasets import Dataset

# Convert the DataFrame into a Hugging Face dataset
dataset = Dataset.from_pandas(data[['input_string']])

# Display the dataset structure
dataset


Dataset({
    features: ['input_string'],
    num_rows: 480
})

In [5]:
from transformers import GPT2Tokenizer
import torch

# Tokenizer setup
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token  # Padding token

# Define tokenization function
def tokenize_function(examples):
    tokenized = tokenizer(examples['input_string'], padding="max_length", truncation=True)
    tokenized['labels'] = tokenized['input_ids'].copy()  # Labels for GPT-2 are just the input shifted
    return tokenized

# Tokenize the dataset and map input_ids to labels
tokenized_data = dataset.map(tokenize_function, batched=True)

# Set the format to return tensors, as required for PyTorch models
tokenized_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/480 [00:00<?, ? examples/s]

In [ ]:
# Import necessary libraries
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset

# Tokenization setup using distilgpt2
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token  # Add padding token if missing

# Load your dataset (ensure you've already preprocessed and tokenized it)
# Assuming tokenized_data exists

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Changed 'evaluation_strategy' to 'eval_strategy' to avoid deprecation warning
    per_device_train_batch_size=2,  # Adjust batch size to prevent memory issues
    gradient_accumulation_steps=8,  # Simulate a larger batch size by accumulating gradients
    num_train_epochs=3,
    weight_decay=0.01
)

# Load the distilgpt2 model (lighter version of GPT-2)
model = GPT2LMHeadModel.from_pretrained("distilgpt2")

# Set up the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    eval_dataset=tokenized_data,  # You may want to split the data for evaluation
)

# Start training
trainer.train()


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
